In [22]:
import re
import sys
import os
# !pip3 install nest_asyncio
# !pip3 install twint
# !pip3 install nltk
#pip install tweet-preprocessor
import nest_asyncio
import string
nest_asyncio.apply()
import twint
import pandas as pd
import nltk
import textblob
from elasticsearch import Elasticsearch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, RegexpTokenizer
from nltk.tokenize import sent_tokenize, PunktSentenceTokenizer, WhitespaceTokenizer
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY)
#p.set_options(p.OPT.URL, p.OPT.EMOJI)
from textblob import TextBlob
# nltk.download('punkt')
nltk.download('stopwords')
%run "candidates.ipynb"

                         0
0      michael mike bennet
1         joseph joe biden
2   michael mike bloomberg
3              cory booker
4            steve bullock
5           pete buttigieg
6            julian castro
7             john delaney
8            tulsi gabbard
9            kamala harris
10           amy klobuchar
11           deval patrick
12          bernie sanders
13              joe sestak
14              tom steyer
15    elizabeth liz warren
16     marianne williamson
17             andrew yang
18            donald trump
19               joe walsh


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dominic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
candidates = ["michael bennet", "joseph biden",
              "michael bloomberg", "cory booker",
              "steve bullock", "pete buttigieg",
              "julian castro", "john delaney",
              "tulsi gabbard", "kamala harris",
              "amy klobuchar", "deval patrick",
              "bernie sanders", "joe sestak",
              "tom steyer", "elizabeth warren",
              "marianne williamson", "andrew yang",
              "donald trump", "joe walsh"]

class candidate:
    def __init__(self, name):
        self.name = name
        self.tweets = pd.DataFrame()
    def tweet_crawler(self, count):
        api = TwitterClient()
        self.tweets = api.get_tweets(self.name, count)


class TwitterClient:
    '''
    Twitter Twint intialization
    '''
    def __init__(self):
        #Intializes the Twint object
        self.twint_Api = twint.Config()

    def preprocess(self, tweets):
        for i in range(len(tweets)):
            tweets.at[i, 'tweet'] = p.clean( tweets.at[i, 'tweet']) 
        return tweets
    
    def clean_tweets(self, data ):
        cleaned_tweets = {}
        newdata = []

        stop_words = stopwords.words('english')
        words = set(nltk.corpus.words.words())

        for token in data.tweet: 
            newdoc = ""
            #print(token)
            seperator = " " 
            token = re.sub('[^A-Za-z0-9]+', ' ', token) #remove special characters
            token = re.sub(r'\d+', '', token) #remove numbers
            #token = re.sub(r'\b\w{1,2}\b', '', token) #remove words with <= 2 characters
            token = " ".join(w for w in nltk.wordpunct_tokenize(token) \
                    # if w.lower() in words or 
                     or not w.isalpha())
            whitespace_token = WhitespaceTokenizer().tokenize( token )
            #wo_stopwords_token = [x for x in whitespace_token
            #                      if not x in stop_words]
            newdoc = seperator.join( (whitespace_token) ).lower()
            #print(newdoc)
            newdata.append(newdoc)
        
        #print(newdata)
        cleaned_tweets['tweet'] = newdata
    
        newtweets = pd.DataFrame(cleaned_tweets)

        return newtweets

    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(tweet)
        return analysis.sentiment.polarity, analysis.sentiment.subjectivity

    def twint_to_pandas(self, col):
        return twint.output.panda.Tweets_df[col]

    def get_tweets(self, search, count):

        #Creates a search string from the list of presidential candidates
        self.twint_Api.Search = search
        self.twint_Api.Limit = count
        self.twint_Api.Format = "Date: {date} | Tweet: {tweet}"
        #self.twint_Api.Location = True
        self.twint_Api.Lang = "en"
        self.twint_Api.Pandas = True
        self.Output = "output.csv"
        # self.twint_Api.Pandas_au = True
        self.twint_Api.Hide_output = True
        #self.twint_Api.Elasticsearch = "http://localhost:9200"
        twint.run.Search(self.twint_Api)

        tweets_df = self.twint_to_pandas(["date", "tweet"])
        
        tweet_sentiment = []
        tweet_subjectivity = []
        
        tweets_df = self.preprocess(tweets_df)
        print(tweets_df)
        self.clean_tweets(tweets_df)
        print(tweets_df)
        
        for tweet in tweets_df.tweet:
            #print(tweet)
            sentiment = self.get_tweet_sentiment(tweet)
            tweet_sentiment.append(sentiment[0])
            tweet_subjectivity.append(sentiment[1])
            #print(tweet)
            
        print(tweet_sentiment)
        tweets_df['Sentiment'] = tweet_sentiment
        return tweets_df

In [ ]:
#test = candidate(candidates[2])
#test.tweet_crawler()

# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

candidates_dict = {}
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

for i in range(len(candidates)):
    candidates_dict[candidates[i]] = []
    d = candidate(c_searchterms[i])
    d.tweet_crawler(10)
    candidates_dict[candidates[i]].append( d.tweets )
    doc = candidates_dict[candidates[i]][0].to_json(orient='index', date_format='iso')
    candidates_dict[candidates[i]][0].to_json("json/" + str(candidates[i]) + ".json", \
                                              orient='index', date_format='iso')
    res = es.index(index="test", doc_type='tweet', body=doc)

    


b'{\n  "name" : "dominic-HP-15-Notebook-PC",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "LMk51hmbSS26QGBPgkpODQ",\n  "version" : {\n    "number" : "7.5.0",\n    "build_flavor" : "default",\n    "build_type" : "deb",\n    "build_hash" : "e9ccaed468e2fac2275a3761849cbee64b39519f",\n    "build_date" : "2019-11-26T01:06:52.518245Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.3.0",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'
                   date                                              tweet
0   2019-12-05 18:02:24  New Hampshire Representative Marjorie Smith En...
1   2019-12-04 12:01:16  Michael Bennet Calls on MLB Commissioner to Sa...
2   2019-12-03 18:10:15  Michael Bennet to Launch Five-Day Tour in New ...
3   2019-12-03 17:20:07  New polling shows most Coloradans don’t know w...
4   2019-12-03 16:45:39  People I would willingly vote

/home/dominic/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                   date                                              tweet
0   2019-12-05 19:16:38  Wow! I'm sure you must think Trump is a wonder...
1   2019-12-05 18:55:27  Gee, and @TheDemocrats thrash @POTUS for less ...
2   2019-12-05 18:53:34  Do you even remember what day it is, Joseph, I...
3   2019-12-05 18:18:53  "[Biden] added, the author [of Burn Pits: the ...
4   2019-12-05 18:04:24  Even though we have our differences, you do de...
5   2019-12-05 17:45:39  Same goes to you, handsy uncle Joseph. You bab...
6   2019-12-05 16:46:26  He shouted out the Joseph R Biden school of pu...
7   2019-12-05 15:55:23  Nice bit of projection there Joseph... Put you...
8   2019-12-05 15:37:15  One Joseph P Biden is the catalyst for that ar...
9   2019-12-05 15:22:20  Concerned voters are calling for all corrupt p...
10  2019-12-05 15:13:26  Sen. Joseph Biden of Delaware, whose campaign ...
11  2019-12-05 14:48:21    A nice FAT tax hike on the rich, right, Joseph?
12  2019-12-05 14:36:56  

                   date                                              tweet
0   2019-12-05 19:32:28  Steve Bullock exits: Will conservative Democra...
1   2019-12-05 19:29:05  Gov. Steve Bullock heard a smorgasbord of conc...
2   2019-12-05 19:20:06  Gov. Steve Bullock heard a smorgasbord of conc...
3   2019-12-05 18:55:32  #AmyKlobuchar endorsed by former Steve Bullock...
4   2019-12-05 18:38:42  New column is up. "So how many times must Stev...
5   2019-12-05 18:02:56  Last night I was trying to come up with a joke...
6   2019-12-05 17:51:00  Steve Bullock exits: Will conservative Democra...
7   2019-12-05 17:11:41  Based on a weighted average, the optimum age i...
8   2019-12-05 15:58:04  The most reasonable candidates from the left, ...
9   2019-12-05 15:40:28  today i am announcing that i am dropping out o...
10  2019-12-05 14:54:48  But she did note that she has hired staffers f...
11  2019-12-05 14:43:07                          What’s a “Steve Bullock?”
12  2019-12-05 14:32:47  

                   date                                              tweet
0   2019-12-05 19:59:59  @JohnDelaney yknow, the internet might hate yo...
1   2019-12-05 19:58:47  I have just received confirmation that John De...
2   2019-12-05 19:58:16                     Delaney Redemption Arc POGGIES
3   2019-12-05 19:56:09  Is this a fake Delaney account or did you fina...
4   2019-12-05 19:55:52  Why do @JoeBiden and @JohnDelaney keep talking...
5   2019-12-05 19:55:38                                           Drop out
6   2019-12-05 19:54:50                             that man? John Delaney
7   2019-12-05 19:53:50  It is not free, even to the insured. What you ...
8   2019-12-05 19:52:22  Wow people really hate a president who has put...
9   2019-12-05 19:51:56              Someone steal your twitter account???
10  2019-12-05 19:51:52  You nailed it. Biden isn't being ambitious at ...
11  2019-12-05 19:46:17  I hate this argument from the right, and I’m o...
12  2019-12-05 19:44:33  